In [2]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.1 MB/s eta 0:00:0000:0100:010m
    torch (>=1.8.*)
           ~~~~~~^

[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [98]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
file_path = "./Traffic_Data_Nationality.xlsx"  # Update with the correct path if needed
xls = pd.ExcelFile(file_path)
# Load the first sheet
df = pd.read_excel(xls, sheet_name="Nationality")

In [166]:
# Handle missing values
df.dropna(inplace=True)

# Define target and predictor variables
target = 'Injury_Severity'
predictors = ['Count', 'Gender', 'Nationality', 'Role', 'Year']


In [167]:
# One-hot encoding for categorical variables
#encode categorical predictor variables (One-Hot Encoding)
df = pd.get_dummies(df,columns=['Nationality','Role','Gender'])

# Standardize numerical feature
scaler = StandardScaler()
df['Year_Scaled'] = scaler.fit_transform(df[['Year']])
df['Count_Scaled'] = scaler.fit_transform(df[['Count']])
Severity = {'Deaths': 1, 'SevereInjury':1, 'SlightInjury':0}
df['Injury_Severity'] = df['Injury_Severity'].map(Severity)

# Concatenate encoded categorical features and scaled numerical feature
final_data = df.drop(['Count','Year'], axis=1)
final_data.head()

,Injury_Severity,Nationality_Foreign,Nationality_G.C.C,Nationality_Not Stated,Nationality_Other Arabs,Nationality_Qataris,Role_Driver,Role_Passenger,Role_Pedestrian,Gender_Female,Gender_Male,Year_Scaled,Count_Scaled
0,1,0,0,0,0,1,1,0,0,0,1,1.512243,-0.263175
1,1,0,1,0,0,0,1,0,0,0,1,1.512243,-0.313336
2,1,0,0,0,1,0,1,0,0,0,1,1.512243,-0.286780
3,1,1,0,0,0,0,1,0,0,0,1,1.512243,-0.145149
4,1,0,0,1,0,0,1,0,0,0,1,1.512243,-0.316286


In [168]:
#Drop one category from each dummy variable to avoid multicollinearity
final_data = final_data.drop(['Nationality_Qataris','Role_Driver','Gender_Female'], axis=1)
final_data.head()

,Injury_Severity,Nationality_Foreign,Nationality_G.C.C,Nationality_Not Stated,Nationality_Other Arabs,Role_Passenger,Role_Pedestrian,Gender_Male,Year_Scaled,Count_Scaled
0,1,0,0,0,0,0,0,1,1.512243,-0.263175
1,1,0,1,0,0,0,0,1,1.512243,-0.313336
2,1,0,0,0,1,0,0,1,1.512243,-0.286780
3,1,1,0,0,0,0,0,1,1.512243,-0.145149
4,1,0,0,1,0,0,0,1,1.512243,-0.316286


In [169]:
# Check for multicollinearity (VIF)
def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data['Feature'] = df.columns
    vif_data['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_data

vif_results = calculate_vif(final_data.drop(columns=[target]))
print("VIF Results:")
print(vif_results)

VIF Results:
                   Feature       VIF
0      Nationality_Foreign  1.482426
1        Nationality_G.C.C  1.417787
2   Nationality_Not Stated  1.302468
3  Nationality_Other Arabs  1.402234
4           Role_Passenger  1.626375
5          Role_Pedestrian  1.703971
6              Gender_Male  1.826278
7              Year_Scaled  1.078041
8             Count_Scaled  1.197734


In [170]:
# Train-test split
X = final_data.drop(columns=[target])
y = final_data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

""" # Polynomial features for interaction terms
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test) """

' # Polynomial features for interaction terms\npoly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)\nX_train_poly = poly.fit_transform(X_train)\nX_test_poly = poly.transform(X_test) '

In [171]:
# Train models
models = {
    'Logistic Regression': LogisticRegression(solver='lbfgs', max_iter=500, class_weight='balanced'),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'),
    'XGBoost': XGBClassifier(eval_metric='mlogloss')
}

In [189]:
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))

Logistic Regression Accuracy: 0.7560
              precision    recall  f1-score   support

           0       0.66      0.59      0.62        71
           1       0.80      0.84      0.82       138

    accuracy                           0.76       209
   macro avg       0.73      0.72      0.72       209
weighted avg       0.75      0.76      0.75       209

Random Forest Accuracy: 0.9234
              precision    recall  f1-score   support

           0       0.91      0.86      0.88        71
           1       0.93      0.96      0.94       138

    accuracy                           0.92       209
   macro avg       0.92      0.91      0.91       209
weighted avg       0.92      0.92      0.92       209

XGBoost Accuracy: 0.9282
              precision    recall  f1-score   support

           0       0.89      0.90      0.90        71
           1       0.95      0.94      0.95       138

    accuracy                           0.93       209
   macro avg       0.92      0.92  

## Cross Validation

In [173]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [174]:
# Create a pipeline that combines polynomial feature generation and logistic regression
logreg_pipeline = make_pipeline(
    #PolynomialFeatures(degree=2,interaction_only=True, include_bias=False),
    LogisticRegression(solver='lbfgs', max_iter=500, class_weight='balanced')
)

# Perform cross-validation
logreg_cv_scores = cross_val_score(logreg_pipeline, X, y, cv=5, scoring='accuracy')

# Print the cross-validation scores and mean accuracy
print(f'Logistic regression cross-validation scores: {logreg_cv_scores}')
print(f'Logistic regression mean accuracy: {logreg_cv_scores.mean():.2f}')

Logistic regression cross-validation scores: [0.84210526 0.77033493 0.80382775 0.83732057 0.83173077]
Logistic regression mean accuracy: 0.82


In [175]:
# Create a pipeline that combines polynomial feature generation and logistic regression
rf_pipeline = make_pipeline(
    #PolynomialFeatures(degree=2,interaction_only=True, include_bias=False),
    RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
)

# Perform cross-validation
rf_cv_scores = cross_val_score(rf_pipeline, X, y, cv=5, scoring='accuracy')

# Print the cross-validation scores and mean accuracy
print(f'Random forest cross-validation scores: {rf_cv_scores}')
print(f'Random forest mean accuracy: {rf_cv_scores.mean():.2f}')

Random forest cross-validation scores: [0.96172249 0.92344498 0.91866029 0.97607656 0.97115385]
Random forest mean accuracy: 0.95


In [176]:
# Create a pipeline that combines polynomial feature generation and logistic regression
xgb_pipeline = make_pipeline(
    #PolynomialFeatures(degree=2,interaction_only=True, include_bias=False),
    XGBClassifier(eval_metric='mlogloss')
)

# Perform cross-validation
xgb_cv_scores = cross_val_score(xgb_pipeline, X, y, cv=5, scoring='accuracy')

# Print the cross-validation scores and mean accuracy
print(f'XGBoost cross-validation scores: {xgb_cv_scores}')
print(f'XGBoost regression mean accuracy: {xgb_cv_scores.mean():.2f}')

XGBoost cross-validation scores: [0.97607656 0.93779904 0.94736842 0.97607656 0.95673077]
XGBoost regression mean accuracy: 0.96


In [183]:
from scipy.stats import norm

# Fit logistic regression
log_reg = LogisticRegression(solver='lbfgs', max_iter=500, class_weight='balanced')
log_reg.fit(X_train, y_train)

# Get feature names after polynomial transformation
feature_names = X_train.columns
# Extract coefficients
coefficients = pd.DataFrame(log_reg.coef_, columns=feature_names, index=[1])

# Calculate standard errors of coefficients
# Standard error = sqrt(diagonal of (X^T * X)^-1)
X_poly_with_intercept = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
cov_matrix = np.linalg.inv(X_poly_with_intercept.T @ X_poly_with_intercept)
standard_errors = np.sqrt(np.diag(cov_matrix))[1:]  # Exclude the intercept term standard error
odds_ratios = np.exp(coefficients)

# Calculate 95% confidence intervals
z_score = norm.ppf(0.975)
conf_intervals = pd.DataFrame({
    'Coefficient': coefficients.values.flatten(),
    'Standard Error': standard_errors,
    '95% CI Lower Bound': coefficients.values.flatten() - z_score * standard_errors,
    '95% CI Upper Bound': coefficients.values.flatten() + z_score * standard_errors,
    'Odds Ratio': np.exp(coefficients.values.flatten())
}, index=feature_names)

# Print coefficients and confidence intervals
print("Logistic Regression Coefficients and 95% Confidence Intervals:")
print(conf_intervals)

Logistic Regression Coefficients and 95% Confidence Intervals:
                         Coefficient  Standard Error  95% CI Lower Bound  \
Nationality_Foreign         1.196366        0.104064            0.992405   
Nationality_G.C.C          -1.189671        0.105140           -1.395741   
Nationality_Not Stated     -1.490987        0.129099           -1.744016   
Nationality_Other Arabs     0.442823        0.103012            0.240924   
Role_Passenger              0.116309        0.085570           -0.051405   
Role_Pedestrian            -1.041886        0.086939           -1.212282   
Gender_Male                 1.055365        0.070847            0.916507   
Year_Scaled                 0.089271        0.036166            0.018387   
Count_Scaled               -7.461648        0.035319           -7.530871   

                         95% CI Upper Bound  Odds Ratio  
Nationality_Foreign                1.400326    3.308072  
Nationality_G.C.C                 -0.983601    0.304321  
Na

In [184]:
conf_intervals.sort_values(by="Odds Ratio", ascending=False)

,Coefficient,Standard Error,95% CI Lower Bound,95% CI Upper Bound,Odds Ratio
Nationality_Foreign,1.196366,0.104064,0.992405,1.400326,3.308072
Gender_Male,1.055365,0.070847,0.916507,1.194223,2.873023
Nationality_Other Arabs,0.442823,0.103012,0.240924,0.644722,1.557097
Role_Passenger,0.116309,0.085570,-0.051405,0.284022,1.123342
Year_Scaled,0.089271,0.036166,0.018387,0.160154,1.093377
Role_Pedestrian,-1.041886,0.086939,-1.212282,-0.871489,0.352789
Nationality_G.C.C,-1.189671,0.105140,-1.395741,-0.983601,0.304321
Nationality_Not Stated,-1.490987,0.129099,-1.744016,-1.237958,0.225150
Count_Scaled,-7.461648,0.035319,-7.530871,-7.392425,0.000575


In [ ]:
conf_intervals.to_csv("./Odds Ratios.csv")

In [178]:
# Feature importance for Random Forest
rf_importances = pd.Series(models['Random Forest'].feature_importances_, index=feature_names).sort_values(ascending=False)
print("Random Forest Feature Importance:")
print(rf_importances)

# Feature importance for XGBoost
xgb_importances = pd.Series(models['XGBoost'].feature_importances_, index=feature_names).sort_values(ascending=False)
print("XGBoost Feature Importance:")
print(xgb_importances)

Random Forest Feature Importance:
Count_Scaled               0.650140
Year_Scaled                0.084503
Gender_Male                0.057467
Nationality_G.C.C          0.053368
Role_Pedestrian            0.043185
Nationality_Not Stated     0.042206
Nationality_Foreign        0.030404
Nationality_Other Arabs    0.022610
Role_Passenger             0.016117
dtype: float64
XGBoost Feature Importance:
Count_Scaled               0.227289
Gender_Male                0.159954
Nationality_G.C.C          0.144220
Nationality_Not Stated     0.126937
Nationality_Foreign        0.126930
Nationality_Other Arabs    0.086517
Role_Pedestrian            0.081116
Role_Passenger             0.037769
Year_Scaled                0.009268
dtype: float32
